Typically you have many tables of data, and you need to combine them to answer the questions that you’re interested in. Collectively, multiple tables of data are called relational data because it is the relations, not just the individual datasets, that are important.

Relations are always defined between a pair of tables. The relations of three or more tables are always a property of the relations between each pair. The figure below shows the relations among the five datasets in `nycflights13` package.

<img src="./figures/transformation/flight_structure.jpg" alt="ds" style="width: 750px;"/>
To work with relational data you need verbs that work with pairs of tables. There are three families of verbs designed to work with relational data:

* **Mutating joins**, which add new variables to one data frame from matching observations in another.

* **Filtering joins**, which filter observations from one data frame based on whether or not they match an observation in the other table.

* **Set operations**, which treat observations as if they were set elements.







# Toy example


```r
band <- tribble(
   ~name,   ~band,  
  "Mick", "Stones",
  "John", "Beetles",
  "Paul", "Beetles"
)

instrument <- tribble(
  ~name, ~plays,
  "John", "guitar",
  "Paul", "bass",
  "Keith", "guitar"
)
```

<img src="./figures/transformation/toy1.jpg" alt="ds" style="width: 750px;"/>


# Join

## Mutating join

```r
left_join(band,instrument, by ="name")
right_join(band,instrument, by ="name")
inner_join(band,instrument, by ="name")
full_join(band,instrument, by ="name")
```


## Your turn
* Which airlines had the largest arrival delays (on average)?

<!-- ```{r echo=FALSE,eval=FALSE}
flights %>% 
  filter(!is.na(arr_delay)) %>%
  left_join(airlines, by = "carrier") %>%
  group_by(name) %>%
  summarize(delay = mean(arr_delay)) %>%
  arrange(desc(delay))


# alternative way
flights %>% 
  filter(!is.na(arr_delay)) %>% 
  group_by(carrier) %>% 
  summarize(delay = mean(arr_delay)) %>% 
  left_join(airlines, by="carrier") %>% 
    arrange(desc(delay))
``` -->


## Different variable names
```r
band <- tribble(
   ~name,   ~band,  
  "Mick", "Stones",
  "John", "Beetles",
  "Paul", "Beetles"
)

instrument2 <- tribble(
  ~artist, ~plays,
  "John", "guitar",
  "Paul", "bass",
  "Keith", "guitar"
)
```

```r
left_join(band,instrument2, by = c("name" = "artist" ))
```

## Your turn
* We can investigate the relationship between distance and the delay.
Join `flights` and `airports` by `dest` and `faa`. 
Then for each name, compute the distance from NYC and the average arr_delay. (use `first()` to get the first value of distance.)
Order by average delay, worst to best.

<!--```{r echo=FALSE,eval=FALSE}
flights %>% 
  filter(!is.na(arr_delay)) %>%
  left_join(airports, by = c("dest" = "faa")) %>%
  group_by(name) %>%
  summarize(distance = first(distance), delay = mean(arr_delay)) %>%
  arrange(desc(delay))
```-->



<!-- ```{r echo=FALSE,eval=FALSE}
flights %>% 
  filter(!is.na(arr_delay)) %>%
  left_join(airports, by = c("dest" = "faa")) %>%
  group_by(name,origin) %>%
  summarize(distance = first(distance), delay = mean(arr_delay)) %>%
  arrange(desc(delay)) %>% 
  filter(name == "Columbia Metropolitan")
```  -->


## Filtering joins

```r
semi_join(band,instrument, by ="name")
anti_join(band,instrument, by ="name")
```

## Your turn
*  How many airports can you fly to direct from New York?
Notice that the column to filter on is named `faa` in the airports dataset and `dest` in the flights dataset.)

<!--```{r echo=FALSE,eval=FALSE}
airports %>%
  semi_join(flights, by = c("faa" = "dest")) %>%
  select(faa,name)
```-->

<img src="./figures/transformation/summary_join.jpg" alt="ds" style="width: 750px;"/>

# Tidy data


```r
table1
table2
table3
table4a
table4b
table5
```

## `gather()`

```r
cases <- tribble(
  ~country, ~"2011", ~"2012",~"2013",
  "FR",  7000, 6900, 7000,
  "DE",  5800, 6000, 6200,
  "US", 15000,14000,13000
)
```

<img src="./figures/transformation/gather1.jpg" alt="ds" style="width: 750px;"/>

```r
cases %>% gather(key = "year", value = "n", 2:4)
```

<img src="./figures/transformation/gather2.jpg" alt="ds" style="width: 750px;"/>

We can use a simlar syntax for selecting the columns.
```r
cases %>% gather(key = "year", value = "n", "2011","2012","2013")
cases %>% gather(key = "year", value = "n", -1)
cases %>% gather(key = "year", value = "n", -country)
cases %>% gather(key = "year", value = "n", starts_with("2"))
```

## Your turn
* Use `gather()` to reorganize table4a into three columns: `country`, `year`, and `cases`.
<!-- ```{r echo=FALSE,eval=FALSE}
table4a %>% gather(key = "year", value = "cases",2,3, convert= TRUE)
``` -->


## `spread()`

```r
pollution <- tribble(
  ~city,   ~size, ~amount, 
  "New York", "large",     23,
  "New York", "small",     14,
  "London", "large",       22,
  "London", "small",       16,
  "Beijing", "large",      121,
  "Beijing", "small",      56
)
```

<img src="./figures/transformation/spread1.jpg" alt="ds" style="width: 750px;"/>

```r
pollution %>% spread(key = size, value = amount)
```

<img src="./figures/transformation/spread2.jpg" alt="ds" style="width: 750px;"/>

## Your turn
* Use `spread()` to reorganize table2 into four columns: `country`, `year`, `cases`, and `population`.
<!--```{r echo=FALSE,eval=FALSE}
table2 %>% spread(key = type, value = count)
```   -->


## `who` data
The data contain
tuberculosis (TB) cases broken down by year, country, age, gender, and diagnosis method from the 2014 World Health Organization Global Tuberculosis Report

`country`, `iso2`, `iso3` - country identifiers

`year` - year

other columns names - encode type of TB case, sex, and age

<img src="./figures/transformation/whocode.jpg" alt="ds" style="width: 750px;"/>

```r
# To avoid a distracting detail during class
names(who) <- str_replace(names(who), "newrel", "new_rel")
```

### Your turn
* Confirm that iso2 and iso3 were redundant with country (Hint: if `iso2` and `iso3` are redundant with country, then, within each country, there should only be one distinct combination of `iso2` and `iso3` values.)

<!--```{r echo=FALSE,eval=FALSE}
who %>% 
  group_by(country) %>% 
  summarize(n_iso = n_distinct(iso2,iso3) ) %>% 
  filter( n_iso !=1 )
``` -->


* Gather the 5th through 60th columns of who into a pair of `key:value` columns named codes and n.
Then select just the `country`, `year`, `codes` and `n` variables.

<!--```{r echo=FALSE,eval=FALSE}
who %>% gather(codes,n, 5:60) %>% 
  select(-iso2,-iso3)
```-->


## `separate`

```r
who %>% gather(codes,n, 5:60) %>% 
  select(-iso2,-iso3) %>% 
  separate(codes,into = c("new", "type", "sexage"), sep = "_")
```


### Your turn
* Separate the sexage column into sex and age columns.
<!--```{r echo=FALSE,eval=FALSE}
who %>% gather(codes,n, 5:60) %>% 
  select(-iso2,-iso3) %>% 
  separate(codes,into = c("new", "type", "sexage"), sep = "_") %>% 
  select(-new) %>% 
  separate(sexage,into = c("sex","age"), sep = 1,convert = TRUE)
```-->


* Calculate rate (in numeric) in `table3`

<!--```{r echo=FALSE,eval=FALSE}
table3 %>% 
  separate(rate,into = c("case","population"),sep="/",convert=TRUE) %>% 
  mutate(rate = case/population)
```-->


## `unite()`
Unites columns into single column by combining cells.
<img src="./figures/transformation/unite.jpg" alt="ds" style="width: 750px;"/>

```r
who %>%
  gather("codes", "n", 5:60) %>%
  select(-iso2, -iso3) %>%
  separate(codes, c("new", "type", "sexage"), sep = "_") %>%
  select(-new) %>%
  separate(sexage, into = c("sex", "age"), sep = 1,convert=TRUE) %>%
  unite("sexage2", sex, age, sep = "-")
```



## Reshaping
Can we calculate the yearly percent of boys (or girls), i.e., male/(male+female)*100?

```r
babynames %>%
  group_by(year, sex) %>% 
  summarize(n = sum(n))
```

### Your turn

* Extend this code to reshape the data. Calculate the percent of male (or female) children by year. Then plot the percent over time.
<!--```{r eval=FALSE}
babynames %>%
  group_by(year, sex) %>% 
  summarize(n = sum(n))
```-->

<!--```{r echo=FALSE,eval=FALSE}
babynames %>%
  group_by(year, sex) %>% 
  summarize(n = sum(n)) %>% 
  spread(key = sex, value = n) %>% 
  mutate(percent = M/(F+M)*100) %>% 
  ggplot()+
  geom_line(aes(x = year, y = percent))
```-->
